In [14]:
import torch
import torch.nn as nn
import torchaudio
from sklearn.metrics import classification_report
from torchaudio.models import wav2vec2_base

class ChordClassifier(nn.Module):
    def __init__(self, pretrained_model):
        super(ChordClassifier, self).__init__()
        
        # Pretrained feature extractor (e.g., wav2vec 2.0 base, or use a custom audio model like YAMNet)
        self.feature_extractor = pretrained_model
        # Freeze pretrained model layers if you don't want to fine-tune it
        for param in self.feature_extractor.parameters():
            param.requires_grad = False
        
        # Your custom layers after feature extraction
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        self.fc1 = nn.Linear(64 * 8 * 69, 128)  # Update these dimensions according to the output shape from conv layers
        self.fc2 = nn.Linear(128, 1)  # Binary classification output (major vs minor)

        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        print(x.shape)

        x = x.squeeze(1)
        print(x.shape)
        # Before passing to the pre-trained model, reshape to (batch_size, time_steps)
        # Collapse the frequency and channel dimensions.
        #input_tensor = mel_spectrogram.view(batch_size, -1)

        # Use pretrained model to extract features
        with torch.no_grad():
            features = self.feature_extractor(x)
        
        # Pass through your convolutional layers
        x = self.relu(self.conv1(features))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        
        # Flatten for fully connected layers
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))  # Sigmoid for binary classification (minor vs major)
        
        return x

    def evaluate(self, dataloader, device):
        self.eval()  # Set model to evaluation mode (deactivates dropout if any)
        all_preds = []
        all_labels = []
    
        # Disable gradient calculations for inference to save memory
        with torch.no_grad():
            for inputs, labels in dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = self(inputs)
                predicted = (outputs > 0.5).float()
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
    
        # Get a classification report
        report = classification_report(all_labels, all_preds, target_names=["Minor", "Major"], zero_division=0)
        print(report)
        return report



In [15]:
from torch.utils.data import Dataset, DataLoader
import os
import torch
import torchaudio.transforms as T
from sklearn.model_selection import train_test_split

class ChordDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths  # List of file paths to .wav files
        self.labels = labels  # List of labels (0 for minor, 1 for major)
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        
        waveform, sample_rate = torchaudio.load(file_path)  # Load the .wav file
        
        # Convert waveform to Mel-spectrogram
        mel_spec_transform = T.MelSpectrogram(sample_rate=sample_rate, n_mels=64)
        mel_spectrogram = mel_spec_transform(waveform)
        
        # Add a channel dimension for the CNN input
        mel_spectrogram = mel_spectrogram
        
        if self.transform:
            mel_spectrogram = self.transform(mel_spectrogram)
        
        return mel_spectrogram, torch.tensor(label, dtype=torch.float32)

# Example usage for DataLoader
def get_dataloader(file_paths, labels, batch_size=16, shuffle=True):
    dataset = ChordDataset(file_paths, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader


In [16]:
def train_model(model, dataloader, num_epochs=10, learning_rate=0.2):
    criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(dataloader):
            # Move data to the appropriate device (GPU or CPU)
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Print statistics
            running_loss += loss.item()
            if i % 10 == 9:  # Print every 10 mini-batches
                print(f'Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss / 10:.4f}')
                running_loss = 0.0

    print('Finished Training')

# Example device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate a pretrained model (e.g., wav2vec2, or any suitable pretrained audio model)
pretrained_model = wav2vec2_base()

# Instantiate your classifier with the pretrained model as an intermediate layer
model = ChordClassifier(pretrained_model).to(device)

# Continue with training, evaluation, etc., as per your previous implementation

# Define the path to the directory containing the .wav files
file_dir = r'C:\Users\rapha\repositories\guitar_hero\data\raw'

# Get a list of all .wav files in the directory
file_paths = [os.path.join(file_dir, f) for f in os.listdir(file_dir) if f.endswith('.wav')]

labels = [0 if 'Minor' in f else 1 for f in os.listdir(file_dir) if f.endswith('.wav')]  # 0 for Minor, 1 for Major

# Split data into train and test sets (80% train, 20% test)
train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42, stratify=labels)

# Get the DataLoader for training and testing sets
train_dataloader = get_dataloader(train_paths, train_labels)
test_dataloader = get_dataloader(test_paths, test_labels)

# Train the model on the training data
train_model(model, train_dataloader, num_epochs=2, learning_rate=0.2)

# Evaluate the model on the test data using the evaluate method from before
model.evaluate(test_dataloader, device)


C:\Users\rapha\anaconda3\Lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (64) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


torch.Size([16, 1, 64, 552])
torch.Size([16, 64, 552])


ValueError: Expected the input Tensor to be 2D (batch, time). Found: [16, 64, 552]